In [1]:
from pyspark import SparkConf, SparkContext
# sc = SparkContext.getOrCreate(conf=SparkConf().setAppName("MyApp").setMaster("yarn"))
conf = SparkConf()
conf.setMaster("yarn")
conf.setAppName("MyApp")
conf.set("spark.executor.instances", "1")
conf.set("spark.executor.cores", "1")
# conf.set("spark.executor.memory", "6g")
sc = SparkContext.getOrCreate(conf=conf)

In [2]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(follower), int(user))

def step(item):
    prev_v, prev_path, next_v = item[0], item[1][0], item[1][1]
    new_path = prev_path + (next_v,)
    return (next_v, new_path)


In [3]:
def shortest_path(v_from, v_to, dataset_path, numPartitions=100):
    edges = sc.textFile(dataset_path).map(parse_edge).cache()
    x = v_from
    d = 0
    paths = sc.parallelize([(x, (v_from,))]).partitionBy(numPartitions)
    i = 1
    found = False
    while True:
        old_v = [item[0] for item in paths.collect()]
        paths = paths.join(edges, numPartitions).map(step)
        paths = paths.filter(lambda item: len(item[1])==i+1).persist()
        i += 1
        if paths.count() == 0:
            break
        if paths.filter(lambda item: item[0]==v_to).count() > 0:
            path = paths.filter(lambda item: item[0]==v_to).collect()[0][1]
            found = True
            return ','.join([str(v) for v in path])
        edges = edges.filter(lambda item: item[0] not in old_v).persist()
            
    if not found:
        print("No valid path found.")
    return None

        

In [4]:
path = shortest_path(12, 34, "/data/twitter/twitter_sample_small.txt", 2)

In [5]:
sc.stop()

In [6]:
print(path)

12,422,53,52,107,20,23,274,34
